### Getting the tools Ready:

In [1]:
import urduhack
#urduhack.download()

import pandas as pd
from urduhack.preprocessing import *
from urduhack.normalization import normalize

#for tokenizing
from nltk.tokenize import TweetTokenizer
import string
import re
pd.set_option("display.max_rows",1000)
pd.set_option("display.max_colwidth",200)

### Custom Functions For Data Cleaning

In [19]:
def text_preprocessing(text):
    """
    This Functions takes a string as input and returns preprocessed form of it in urdu language.
    """
    # removing digits and english letters
    Text=re.sub(r"[a-zA-z0-9]+", "", text.lower())
    
    #removing \n (new line tag)
    Text=Text.replace("\n","") 
    
    #using urduhack's functions for data preprocessing 
    Text=normalize_whitespace(Text)
    Text=remove_punctuation(Text)
    Text=remove_accents(Text)
    Text=replace_urls(Text)
    Text=replace_emails(Text)
    Text=replace_phone_numbers(Text)
    Text=replace_numbers(Text)
    return Text

In [3]:
def normalization(text):
    """
    This Functions takes a string as input and returns normalized, consistent and cleaned form of it in urdu language.
    """
    
    #using urduhack's functions to normalize text. 
    #refer to docs here: https://docs.urduhack.com/en/stable/reference/normalization.html
    Text=normalize(text)
    
    #removing irregular whitescapes/empty srings/larger strings and imputing them with NaN
    Text=Text.strip()
    if len(Text)==0 or len(Text)> 150:
        Text=Text.replace("", "NaN")
        Text=Text.replace(Text,"NaN")
        return float(Text)
    else:
        return Text

### Data Preprocessing

In [4]:
#reading from csv (scraped from web)
df=pd.read_csv("Anti-Army.csv")

In [5]:
df.head()

,user_name,tweets
0,Ashraf Ali Advocate AAA,یہ تو ہونا ہی تھا ،،،، اور مانگو بزدل فوج سے رسیدیں!
1,PRINCE HAMAD BIN JASSIM (قطری شہزادہ),پٹواری ہو جاہل جھوٹا اور فوج پر بھونکنے والا کتا نہ ہو نہ ممکن پہلے تو یہ تصویر افغانستان کی ہے دوسرا جب نمبر ون انکو پکڑتی ہے تو تیر کنجری باجی اسکی ڈی پی لگا لیتی ہے مسنگ پرسن کے نام پر
2,مرشد Murshid,Quote Tweet\nKamran Khan\n@AajKamranKhan\n · 2h\nپشاور میں مذہبی مدرسے پر دہش گرد حملہ ہمیں بھارتی انٹیلیجنس بذریعہ اجیت ڈوول مسلسل دھمکیاں یاد دلاتا ہے دشمن ہم ہر حملہ آور ہے! سیاسی قیادت اپنے د...
3,Usman Ali Farooqi,فوج اور کمانڈرز کا کام ہیں عوام کا حفاظت کرنا نہ کہ صرف تھرٹ جاری کرنا۔۔۔۔۔۔۔ یہ واقعہ APAسے زیادہ اہم ہے ۔۔دیکھتے کہ آب نیشنل ایکشن فلان کیا کر تا ہے۔۔۔۔
4,سُندس,*بھارتی فوج۔۔۔


In [6]:
#inspecting the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3565 entries, 0 to 3564
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_name  3565 non-null   object
 1   tweets     3565 non-null   object
dtypes: object(2)
memory usage: 55.8+ KB


In [7]:
#droping non-useful columns
df.drop(["user_name"],axis=1,inplace=True)

In [8]:
df.head()

,tweets
0,یہ تو ہونا ہی تھا ،،،، اور مانگو بزدل فوج سے رسیدیں!
1,پٹواری ہو جاہل جھوٹا اور فوج پر بھونکنے والا کتا نہ ہو نہ ممکن پہلے تو یہ تصویر افغانستان کی ہے دوسرا جب نمبر ون انکو پکڑتی ہے تو تیر کنجری باجی اسکی ڈی پی لگا لیتی ہے مسنگ پرسن کے نام پر
2,Quote Tweet\nKamran Khan\n@AajKamranKhan\n · 2h\nپشاور میں مذہبی مدرسے پر دہش گرد حملہ ہمیں بھارتی انٹیلیجنس بذریعہ اجیت ڈوول مسلسل دھمکیاں یاد دلاتا ہے دشمن ہم ہر حملہ آور ہے! سیاسی قیادت اپنے د...
3,فوج اور کمانڈرز کا کام ہیں عوام کا حفاظت کرنا نہ کہ صرف تھرٹ جاری کرنا۔۔۔۔۔۔۔ یہ واقعہ APAسے زیادہ اہم ہے ۔۔دیکھتے کہ آب نیشنل ایکشن فلان کیا کر تا ہے۔۔۔۔
4,*بھارتی فوج۔۔۔


##### Cleaning and Normalizing

In [20]:
df["cleaned"]=df["tweets"].apply(lambda x: text_preprocessing(x))

In [21]:
df.head()

,tweets,cleaned,normalized,tokenized
0,یہ تو ہونا ہی تھا ،،،، اور مانگو بزدل فوج سے رسیدیں!,یہ تو ہونا ہی تھا اور مانگو بزدل فوج سے رسیدیں,یہ تو ہونا ہی تھا اور مانگو بزدل فوج سے رسیدیں,"[یہ, تو, ہونا, ہی, تھا, اور, مانگو, بزدل, فوج, سے, رسیدیں]"
3,فوج اور کمانڈرز کا کام ہیں عوام کا حفاظت کرنا نہ کہ صرف تھرٹ جاری کرنا۔۔۔۔۔۔۔ یہ واقعہ APAسے زیادہ اہم ہے ۔۔دیکھتے کہ آب نیشنل ایکشن فلان کیا کر تا ہے۔۔۔۔,فوج اور کمانڈرز کا کام ہیں عوام کا حفاظت کرنا نہ کہ صرف تھرٹ جاری کرنا یہ واقعہ سے زیادہ اہم ہے دیکھتے کہ آب نیشنل ایکشن فلان کیا کر تا ہے,فوج اور کمانڈرز کا کام ہیں عوام کا حفاظت کرنا نہ کہ صرف تھرٹ جاری کرنا یہ واقعہ سے زیادہ اہم ہے دیکھتے کہ آب نیشنل ایکشن فلان کیا کر تا ہے,"[فوج, اور, کمانڈرز, کا, کام, ہیں, عوام, کا, حفاظت, کرنا, نہ, کہ, صرف, تھرٹ, جاری, کرنا, یہ, واقعہ, سے, زیادہ, اہم, ہے, دیکھتے, کہ, آب, نیشنل, ایکشن, فلان, کیا, کر, تا, ہے]"
4,*بھارتی فوج۔۔۔,بھارتی فوج,بھارتی فوج,"[بھارتی, فوج]"
5,کچھ مثبت باتیں کریں پاکستان کے حق میں، cool نظر آنے کے لیے دشمن ایجنٹوں کی ہاں میں ہاں نہ ملائیں، اپنی فوج کی ہمت بڑ ھائیں,کچھ مثبت باتیں کریں پاکستان کے حق میں نظر آنے کے لیے دشمن ایجنٹوں کی ہاں میں ہاں نہ ملائیں اپنی فوج کی ہمت بڑ ھائیں,کچھ مثبت باتیں کریں پاکستان کے حق میں نظر آنے کے لیے دشمن ایجنٹوں کی ہاں میں ہاں نہ ملائیں اپنی فوج کی ہمت بڑ ھائیں,"[کچھ, مثبت, باتیں, کریں, پاکستان, کے, حق, میں, نظر, آنے, کے, لیے, دشمن, ایجنٹوں, کی, ہاں, میں, ہاں, نہ, ملائیں, اپنی, فوج, کی, ہمت, بڑ, ھائیں]"
8,فوج بھی اور کیا کر سکتی ہے\nہاں نغمہ بھی ریلیز کر سکتی ہے,فوج بھی اور کیا کر سکتی ہےہاں نغمہ بھی ریلیز کر سکتی ہے,فوج بھی اور کیا کر سکتی ہےہاں نغمہ بھی ریلیز کر سکتی ہے,"[فوج, بھی, اور, کیا, کر, سکتی, ہےہاں, نغمہ, بھی, ریلیز, کر, سکتی, ہے]"


In [22]:
df["normalized"]=df["cleaned"].apply(lambda x : normalization(x))

In [23]:
df[:200]

,tweets,cleaned,normalized,tokenized
0,یہ تو ہونا ہی تھا ،،،، اور مانگو بزدل فوج سے رسیدیں!,یہ تو ہونا ہی تھا اور مانگو بزدل فوج سے رسیدیں,یہ تو ہونا ہی تھا اور مانگو بزدل فوج سے رسیدیں,"[یہ, تو, ہونا, ہی, تھا, اور, مانگو, بزدل, فوج, سے, رسیدیں]"
3,فوج اور کمانڈرز کا کام ہیں عوام کا حفاظت کرنا نہ کہ صرف تھرٹ جاری کرنا۔۔۔۔۔۔۔ یہ واقعہ APAسے زیادہ اہم ہے ۔۔دیکھتے کہ آب نیشنل ایکشن فلان کیا کر تا ہے۔۔۔۔,فوج اور کمانڈرز کا کام ہیں عوام کا حفاظت کرنا نہ کہ صرف تھرٹ جاری کرنا یہ واقعہ سے زیادہ اہم ہے دیکھتے کہ آب نیشنل ایکشن فلان کیا کر تا ہے,فوج اور کمانڈرز کا کام ہیں عوام کا حفاظت کرنا نہ کہ صرف تھرٹ جاری کرنا یہ واقعہ سے زیادہ اہم ہے دیکھتے کہ آب نیشنل ایکشن فلان کیا کر تا ہے,"[فوج, اور, کمانڈرز, کا, کام, ہیں, عوام, کا, حفاظت, کرنا, نہ, کہ, صرف, تھرٹ, جاری, کرنا, یہ, واقعہ, سے, زیادہ, اہم, ہے, دیکھتے, کہ, آب, نیشنل, ایکشن, فلان, کیا, کر, تا, ہے]"
4,*بھارتی فوج۔۔۔,بھارتی فوج,بھارتی فوج,"[بھارتی, فوج]"
5,کچھ مثبت باتیں کریں پاکستان کے حق میں، cool نظر آنے کے لیے دشمن ایجنٹوں کی ہاں میں ہاں نہ ملائیں، اپنی فوج کی ہمت بڑ ھائیں,کچھ مثبت باتیں کریں پاکستان کے حق میں نظر آنے کے لیے دشمن ایجنٹوں کی ہاں میں ہاں نہ ملائیں اپنی فوج کی ہمت بڑ ھائیں,کچھ مثبت باتیں کریں پاکستان کے حق میں نظر آنے کے لیے دشمن ایجنٹوں کی ہاں میں ہاں نہ ملائیں اپنی فوج کی ہمت بڑ ھائیں,"[کچھ, مثبت, باتیں, کریں, پاکستان, کے, حق, میں, نظر, آنے, کے, لیے, دشمن, ایجنٹوں, کی, ہاں, میں, ہاں, نہ, ملائیں, اپنی, فوج, کی, ہمت, بڑ, ھائیں]"
8,فوج بھی اور کیا کر سکتی ہے\nہاں نغمہ بھی ریلیز کر سکتی ہے,فوج بھی اور کیا کر سکتی ہےہاں نغمہ بھی ریلیز کر سکتی ہے,فوج بھی اور کیا کر سکتی ہےہاں نغمہ بھی ریلیز کر سکتی ہے,"[فوج, بھی, اور, کیا, کر, سکتی, ہےہاں, نغمہ, بھی, ریلیز, کر, سکتی, ہے]"
11,اور مانگو فوج سے رسیدیں!,اور مانگو فوج سے رسیدیں,اور مانگو فوج سے رسیدیں,"[اور, مانگو, فوج, سے, رسیدیں]"
12,جب تک فوج ہے ان کا باپ بھی الگ نہیں کر سکتا پہلے بڑے بڑے سردار آئے اب تو زمانہ جدید ہو گیا اور مشکل ہے,جب تک فوج ہے ان کا باپ بھی الگ نہیں کر سکتا پہلے بڑے بڑے سردار آئے اب تو زمانہ جدید ہو گیا اور مشکل ہے,جب تک فوج ہے ان کا باپ بھی الگ نہیں کر سکتا پہلے بڑے بڑے سردار آئے اب تو زمانہ جدید ہو گیا اور مشکل ہے,"[جب, تک, فوج, ہے, ان, کا, باپ, بھی, الگ, نہیں, کر, سکتا, پہلے, بڑے, بڑے, سردار, آئے, اب, تو, زمانہ, جدید, ہو, گیا, اور, مشکل, ہے]"
13,پہلے دہشت گردوں کو پالنے اور رہا کرنے والی فوج سے تو قانون پر عمل کروالو۔۔۔۔,پہلے دہشت گردوں کو پالنے اور رہا کرنے والی فوج سے تو قانون پر عمل کروالو,پہلے دہشت گردوں کو پالنے اور رہا کرنے والی فوج سے تو قانون پر عمل کروالو,"[پہلے, دہشت, گردوں, کو, پالنے, اور, رہا, کرنے, والی, فوج, سے, تو, قانون, پر, عمل, کروالو]"
14,یہ حملے ہمیشہ سے فوج کرتی ہیں اور تمھارے جیسے دلال لوگوں کی قتل پر سیاست کرتے ہیں،، احسان اللہ احسان کو فوج نے رہا کیا یہ سب کچھ کروانے کیلئے,یہ حملے ہمیشہ سے فوج کرتی ہیں اور تمھارے جیسے دلال لوگوں کی قتل پر سیاست کرتے ہیں احسان اللہ احسان کو فوج نے رہا کیا یہ سب کچھ کروانے کیلئے,یہ حملے ہمیشہ سے فوج کرتی ہیں اور تمھارے جیسے دلال لوگوں کی قتل پر سیاست کرتے ہیں احسان اللہ احسان کو فوج نے رہا کیا یہ سب کچھ کروانے کیلئے,"[یہ, حملے, ہمیشہ, سے, فوج, کرتی, ہیں, اور, تمھارے, جیسے, دلال, لوگوں, کی, قتل, پر, سیاست, کرتے, ہیں, احسان, اللہ, احسان, کو, فوج, نے, رہا, کیا, یہ, سب, کچھ, کروانے, کیلئے]"
15,Quote Tweet\nNasir Butt\n@nasirbuttuk\n · 2h\nیا اللہ پاکستان کی حفاظت فرما۔میرے پاکستان کو غیر محفوظ کر نے والے کرداروں کو کیفر کدار تک پہنچا۔,یا اللہ پاکستان کی حفاظت فرمامیرے پاکستان کو غیر محفوظ کر نے والے کرداروں کو کیفر کدار تک پہنچا,یا اللہ پاکستان کی حفاظت فرمامیرے پاکستان کو غیر محفوظ کر نے والے کرداروں کو کیفر کدار تک پہنچا,"[یا, اللہ, پاکستان, کی, حفاظت, فرمامیرے, پاکستان, کو, غیر, محفوظ, کر, نے, والے, کرداروں, کو, کیفر, کدار, تک, پہنچا]"


In [24]:
#inspecting for missing Values 
df.isna().sum()

tweets        0
cleaned       0
normalized    0
tokenized     0
dtype: int64

In [25]:
#droping missing rows
df.dropna(inplace=True,axis=0)

In [26]:
#inspecting again
df.isna().sum()

tweets        0
cleaned       0
normalized    0
tokenized     0
dtype: int64

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1553 entries, 0 to 3560
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweets      1553 non-null   object
 1   cleaned     1553 non-null   object
 2   normalized  1553 non-null   object
 3   tokenized   1553 non-null   object
dtypes: object(4)
memory usage: 60.7+ KB


In [28]:
#tokenizing cleaned text for vectorizing
tknzr = TweetTokenizer()
df["tokenized"]=df.normalized.apply(lambda x : tknzr.tokenize(x))

In [29]:
df[:300]

,tweets,cleaned,normalized,tokenized
0,یہ تو ہونا ہی تھا ،،،، اور مانگو بزدل فوج سے رسیدیں!,یہ تو ہونا ہی تھا اور مانگو بزدل فوج سے رسیدیں,یہ تو ہونا ہی تھا اور مانگو بزدل فوج سے رسیدیں,"[یہ, تو, ہونا, ہی, تھا, اور, مانگو, بزدل, فوج, سے, رسیدیں]"
3,فوج اور کمانڈرز کا کام ہیں عوام کا حفاظت کرنا نہ کہ صرف تھرٹ جاری کرنا۔۔۔۔۔۔۔ یہ واقعہ APAسے زیادہ اہم ہے ۔۔دیکھتے کہ آب نیشنل ایکشن فلان کیا کر تا ہے۔۔۔۔,فوج اور کمانڈرز کا کام ہیں عوام کا حفاظت کرنا نہ کہ صرف تھرٹ جاری کرنا یہ واقعہ سے زیادہ اہم ہے دیکھتے کہ آب نیشنل ایکشن فلان کیا کر تا ہے,فوج اور کمانڈرز کا کام ہیں عوام کا حفاظت کرنا نہ کہ صرف تھرٹ جاری کرنا یہ واقعہ سے زیادہ اہم ہے دیکھتے کہ آب نیشنل ایکشن فلان کیا کر تا ہے,"[فوج, اور, کمانڈرز, کا, کام, ہیں, عوام, کا, حفاظت, کرنا, نہ, کہ, صرف, تھرٹ, جاری, کرنا, یہ, واقعہ, سے, زیادہ, اہم, ہے, دیکھتے, کہ, آب, نیشنل, ایکشن, فلان, کیا, کر, تا, ہے]"
4,*بھارتی فوج۔۔۔,بھارتی فوج,بھارتی فوج,"[بھارتی, فوج]"
5,کچھ مثبت باتیں کریں پاکستان کے حق میں، cool نظر آنے کے لیے دشمن ایجنٹوں کی ہاں میں ہاں نہ ملائیں، اپنی فوج کی ہمت بڑ ھائیں,کچھ مثبت باتیں کریں پاکستان کے حق میں نظر آنے کے لیے دشمن ایجنٹوں کی ہاں میں ہاں نہ ملائیں اپنی فوج کی ہمت بڑ ھائیں,کچھ مثبت باتیں کریں پاکستان کے حق میں نظر آنے کے لیے دشمن ایجنٹوں کی ہاں میں ہاں نہ ملائیں اپنی فوج کی ہمت بڑ ھائیں,"[کچھ, مثبت, باتیں, کریں, پاکستان, کے, حق, میں, نظر, آنے, کے, لیے, دشمن, ایجنٹوں, کی, ہاں, میں, ہاں, نہ, ملائیں, اپنی, فوج, کی, ہمت, بڑ, ھائیں]"
8,فوج بھی اور کیا کر سکتی ہے\nہاں نغمہ بھی ریلیز کر سکتی ہے,فوج بھی اور کیا کر سکتی ہےہاں نغمہ بھی ریلیز کر سکتی ہے,فوج بھی اور کیا کر سکتی ہےہاں نغمہ بھی ریلیز کر سکتی ہے,"[فوج, بھی, اور, کیا, کر, سکتی, ہےہاں, نغمہ, بھی, ریلیز, کر, سکتی, ہے]"
11,اور مانگو فوج سے رسیدیں!,اور مانگو فوج سے رسیدیں,اور مانگو فوج سے رسیدیں,"[اور, مانگو, فوج, سے, رسیدیں]"
12,جب تک فوج ہے ان کا باپ بھی الگ نہیں کر سکتا پہلے بڑے بڑے سردار آئے اب تو زمانہ جدید ہو گیا اور مشکل ہے,جب تک فوج ہے ان کا باپ بھی الگ نہیں کر سکتا پہلے بڑے بڑے سردار آئے اب تو زمانہ جدید ہو گیا اور مشکل ہے,جب تک فوج ہے ان کا باپ بھی الگ نہیں کر سکتا پہلے بڑے بڑے سردار آئے اب تو زمانہ جدید ہو گیا اور مشکل ہے,"[جب, تک, فوج, ہے, ان, کا, باپ, بھی, الگ, نہیں, کر, سکتا, پہلے, بڑے, بڑے, سردار, آئے, اب, تو, زمانہ, جدید, ہو, گیا, اور, مشکل, ہے]"
13,پہلے دہشت گردوں کو پالنے اور رہا کرنے والی فوج سے تو قانون پر عمل کروالو۔۔۔۔,پہلے دہشت گردوں کو پالنے اور رہا کرنے والی فوج سے تو قانون پر عمل کروالو,پہلے دہشت گردوں کو پالنے اور رہا کرنے والی فوج سے تو قانون پر عمل کروالو,"[پہلے, دہشت, گردوں, کو, پالنے, اور, رہا, کرنے, والی, فوج, سے, تو, قانون, پر, عمل, کروالو]"
14,یہ حملے ہمیشہ سے فوج کرتی ہیں اور تمھارے جیسے دلال لوگوں کی قتل پر سیاست کرتے ہیں،، احسان اللہ احسان کو فوج نے رہا کیا یہ سب کچھ کروانے کیلئے,یہ حملے ہمیشہ سے فوج کرتی ہیں اور تمھارے جیسے دلال لوگوں کی قتل پر سیاست کرتے ہیں احسان اللہ احسان کو فوج نے رہا کیا یہ سب کچھ کروانے کیلئے,یہ حملے ہمیشہ سے فوج کرتی ہیں اور تمھارے جیسے دلال لوگوں کی قتل پر سیاست کرتے ہیں احسان اللہ احسان کو فوج نے رہا کیا یہ سب کچھ کروانے کیلئے,"[یہ, حملے, ہمیشہ, سے, فوج, کرتی, ہیں, اور, تمھارے, جیسے, دلال, لوگوں, کی, قتل, پر, سیاست, کرتے, ہیں, احسان, اللہ, احسان, کو, فوج, نے, رہا, کیا, یہ, سب, کچھ, کروانے, کیلئے]"
15,Quote Tweet\nNasir Butt\n@nasirbuttuk\n · 2h\nیا اللہ پاکستان کی حفاظت فرما۔میرے پاکستان کو غیر محفوظ کر نے والے کرداروں کو کیفر کدار تک پہنچا۔,یا اللہ پاکستان کی حفاظت فرمامیرے پاکستان کو غیر محفوظ کر نے والے کرداروں کو کیفر کدار تک پہنچا,یا اللہ پاکستان کی حفاظت فرمامیرے پاکستان کو غیر محفوظ کر نے والے کرداروں کو کیفر کدار تک پہنچا,"[یا, اللہ, پاکستان, کی, حفاظت, فرمامیرے, پاکستان, کو, غیر, محفوظ, کر, نے, والے, کرداروں, کو, کیفر, کدار, تک, پہنچا]"
